# Robyn Budget Allocator Demo

This notebook demonstrates how to use the Python implementation of Robyn's budget allocator.
It shows how to:
1. Load and prepare data
2. Configure the allocator
3. Run optimization scenarios
4. Analyze and visualize results

## Step 1: Load Exported R Data

In [ ]:
## Step 1: Setup and Import
import sys
import os
import pandas as pd
import numpy as np
from typing import Dict, Any, Union, List
import matplotlib.pyplot as plt

# Add Robyn to path
sys.path.append("/Users/yijuilee/robynpy_release_reviews/Robyn/python/src")

# Import necessary Robyn classes
from robyn.data.entities.mmmdata import MMMData
from robyn.modeling.entities.modeloutputs import ModelOutputs
from robyn.data.entities.hyperparameters import Hyperparameters
from robyn.allocator.entities.enums import OptimizationScenario, ConstrMode

from utils.data_mapper import load_data_from_json, import_input_collect, import_output_collect, import_output_models

In [ ]:
# Load data from JSON exported from R
raw_input_collect = load_data_from_json(
    "/Users/yijuilee/robynpy_release_reviews/Robyn/python/src/tutorials/data/Allocator_InputCollect.json"
)
raw_output_collect = load_data_from_json(
    "/Users/yijuilee/robynpy_release_reviews/Robyn/python/src/tutorials/data/Allocator_OutputCollect.json"
)
raw_output_models = load_data_from_json(
    "/Users/yijuilee/robynpy_release_reviews/Robyn/python/src/tutorials/data/Allocator_OutputModels.json"
)

# Convert R data to Python objects
r_input_collect = import_input_collect(raw_input_collect)
r_output_collect = import_output_collect(raw_output_collect)
python_model_outputs = import_output_models(raw_output_models)

# Extract individual components
mmm_data = r_input_collect["mmm_data"]
featurized_mmm_data = r_input_collect["featurized_mmm_data"]
holidays_data = r_input_collect["holidays_data"]
model_outputs = python_model_outputs
hyperparameters = r_input_collect["hyperparameters"]

## Step 2: Set up Budget Allocator

Initialize the budget allocator with the selected model and data.

In [ ]:
for solid in r_output_collect["pareto_result"].result_hyp_param["solID"]:
    print(f"Solution ID: {solid}")

In [ ]:
from robyn.new_allocator.budget_allocator import BudgetAllocator

# Select a model from the results
select_model = "1_65_6"  # Example model ID
# Initialize budget allocator
allocator = BudgetAllocator(
    input_collect=mmm_data, output_collect=model_outputs, select_model=select_model, hyperparameters=hyperparameters
)

print(hyperparameters)

## Step 3: Configure Allocation Settings

Set up constraints and parameters for optimization.

## Step 4: Run Different Optimization Scenarios

### Scenario 1: Default Max Response

In [ ]:
# Similar to Example 1 in demo.R
result1 = allocator.run_allocation(
    scenario="max_response",
    channel_constraints_low=0.7,
    channel_constraints_up=[1.2, 1.5, 1.5, 1.5, 1.5],
    date_range="all",
)

# Print results
print(result1)

In [ ]:
print(result1)

### Scenario 2: Max Response with Custom Settings

### Scenario 3: Target Efficiency
Optimize allocation based on target ROI/CPA.

### Scenario 4: Custom Target Efficiency

## Step 5: Visualize Results (One Pager)

Plot the optimization results and response curves.

In [ ]:
# Before creating plots, add these debug lines
print("\nDataFrame columns:", result1.dt_optim_out.columns)
print("\nDataFrame head:\n", result1.dt_optim_out.head())

In [ ]:
date_range = (mmm_data.data[mmm_data.mmmdata_spec.date_var].min(), mmm_data.data[mmm_data.mmmdata_spec.date_var].max())
print(date_range)

In [ ]:
from robyn.new_allocator.allocation_plotter import AllocationPlotter
from robyn.new_allocator.optimization.objective_function import ObjectiveFunction
from robyn.data.entities.mmmdata import MMMData, MMMDataSpec

# Create plots
plotter = AllocationPlotter()

# Get date range
date_range = (mmm_data.data[mmm_data.mmmdata_spec.date_var].min(), mmm_data.data[mmm_data.mmmdata_spec.date_var].max())

# Prepare response curves data
plot_data = plotter._prepare_response_curves_data(
    channels=allocator.paid_media_vars,
    allocation_df=result1.dt_optim_out,
    objective_function=allocator.objective_function,
)

# Create onepager plot
fig = plotter.create_onepager(
    dt_optim_out=result1.dt_optim_out,
    plot_data=plot_data,
    scenario=result1.scenario,
    date_range=date_range,
    interval_type="Week",
)

plt.show()